# Кросс-валидация ARIMA (пакет [`pmdarima`](https://alkaline-ml.com/pmdarima/index.html#))

In [ ]:
import numpy as np
import pandas as pd

import pmdarima as pm
from pmdarima import model_selection
from sklearn.metrics import mean_squared_error as mse 

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) месячные данные по Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity (Symbol [`GS10`](https://fred.stlouisfed.org/series/GS10)) с 2000-01-01 по 2023-12-31 и создадим датафрейм `rate`


In [ ]:
y = web.DataReader(name='GS10', data_source='fred', start='2000-01-01', end='2023-12-31')

## Валидация через разделение на обучающую и тестовую выборки

Используем метод [`train_test_split`](http://alkaline-ml.com/pmdarima/modules/generated/pmdarima.model_selection.train_test_split.html#pmdarima.model_selection.train_test_split) из пакета `pmdarima`

Разберём на примере ARIMA(2,1,3) без сноса

In [ ]:
# специфицируем модель
arima = pm.ARIMA(order=(2,1,3), trend='n')

# разбиваем выбору на обучающую (первые 80%) и тестовую
train, test = model_selection.train_test_split(y, train_size=0.8)
# Обучаем модель на тестовом множестве
arima.fit(train)
# прогнозируем для тестовых наблюдений
forecasts = arima.predict(n_periods=test.shape[0])

# считаем метрику MSE
mse(test, forecasts)

In [ ]:
# Визуализация прогнозов
preds = arima.predict(n_periods=test.shape[0], return_conf_int=False)

plt.plot(preds)
plt.plot(train)
plt.plot(test)
plt.legend(['forecasts', 'train', 'test'])

plt.show()


## Валидация методом k-Fold (расширяем обучающую выборку)

Используем метод [`RollingForecastCV`](http://alkaline-ml.com/pmdarima/modules/generated/pmdarima.model_selection.RollingForecastCV.html#pmdarima.model_selection.RollingForecastCV) из пакета `pmdarima`

Для вычисления метрики для каждого фолда используем метод [`cross_val_score`](http://alkaline-ml.com/pmdarima/modules/generated/pmdarima.model_selection.cross_val_score.html#pmdarima.model_selection.cross_val_score)

Разберём на примере ARIMA(2,1,3) без сноса

In [ ]:
# специфицируем модель
arima = pm.ARIMA(order=(2,1,3), trend='n')

# специфицируем метод кросс-валидации: h - горизонт прогнозирования, step - шаг увеличения обучающей выборки.
# по умолчанию начинаем с 1/3 выборки
cv = model_selection.RollingForecastCV(h=10, step=10)

# считаем метрику для прогнозов (MSE) для каждой итерации (verbose выдаёт отчёт по каждой итерации)
scores = model_selection.cross_val_score(estimator=arima, y=y, cv=cv, scoring='mean_squared_error', verbose=2)

# средняя ошибка
np.average(scores)

In [ ]:
cv = model_selection.RollingForecastCV(h=10, step=10, initial=150)

predictions = model_selection.cross_val_predict(arima, y, cv=cv, verbose=0, averaging="mean")

n_test = predictions.shape[0]

predictions = pd.DataFrame(predictions, index=y.index[-n_test:])

plt.plot(predictions)
plt.plot(y)

plt.show

## Валидация методом k-Fold (скользящая обучающая выборка)

Используем метод [`SlidingWindowForecastCV`](http://alkaline-ml.com/pmdarima/modules/generated/pmdarima.model_selection.SlidingWindowForecastCV.html#pmdarima.model_selection.SlidingWindowForecastCV) из пакета `pmdarima`

Для вычисления метрики для каждого фолда используем метод [`cross_val_score`](http://alkaline-ml.com/pmdarima/modules/generated/pmdarima.model_selection.cross_val_score.html#pmdarima.model_selection.cross_val_score)

Разберём на примере ARIMA(2,1,3) без сноса


In [ ]:
# специфицируем модель
arima = pm.ARIMA(order=(2,0,3), with_intercept=True, trend=None)

# специфицируем метод кросс-валидации: h - горизонт прогнозирования, step - шаг увеличения обучающей выборки.
# по умолчанию начинаем с 1/5 выборки
cv = model_selection.SlidingWindowForecastCV(h=10, step=10, window_size=150)

# считаем метрику для прогнозов (MSE) для каждой итерации (verbose выдаёт отчёт по каждой итерации)
scores = model_selection.cross_val_score(estimator=arima, y=y, cv=cv, scoring='mean_squared_error', verbose=2)

# средняя ошибка
np.average(scores)

In [ ]:
# специфицируем метод кросс-валидации: h - горизонт прогнозирования, step - шаг увеличения обучающей выборки.
# по умолчанию начинаем с 1/5 выборки
cv = model_selection.SlidingWindowForecastCV(h=10, step=10, window_size=150)

# Вычисляем прогноз
predictions = model_selection.cross_val_predict(arima, y, cv=cv, verbose=0, averaging="mean")

n_test = predictions.shape[0]

predictions = pd.DataFrame(predictions, index=y.index[-n_test:])

plt.plot(predictions)
plt.plot(y)

plt.show